In [1]:
import tensorflow as tf

/devkit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### 一. 什么是DataFlow Graph
1. DataFlow是tensorflow的并行计算模型, 顶点代表操作, 边代表消耗或产出的tensor

#### 二. 隐士创建DataFlow Graph
1. 所有创建的tensor都会被自动加入默认的隐式创建的graph. 即以下tensorflow的api, 会同时创建图的顶点(操作)和边(输出张量)  
  1. `tf.constant(42.0)` : 创建单一操作`tf.Operation`,产生标量42.0,并加入默认图
  3. `tf.matmul(x, y)` : 创建单一操作`tf.Operation`, 计算x和y的矩阵乘组作为输出张量, 并加入默认图
  4. `tf.Variable(0)` : 创建操作`tf.Operation`,存储一个可写张量, 在调用`tf.Session.run`之前.`tf.Variable` 对象有 `assign` 和`assign_add`方法创建`tf.Operation`对象(运算).
  5. `tf.train.Optimizer.minimize` : 增加`tf.Operation`和`tf.Tensor`到默认图, 并计算其梯度

#### 三. 命名空间
1. 每个操作都有别名, 可以再声明tensor时指定相应操作的别名  
```python
tf.constant(0, name="c")
```
2. 使用`tf.name_scope("ns")`指定命名空间, 这里面定义的操作命名都有该命名空间的前缀

In [2]:
c_0 = tf.constant(0, name="c")  # => operation named "c"
print('c_0:',c_0)
# Already-used names will be "uniquified".
c_1 = tf.constant(2, name="c")  # => operation named "c_1"
print('c_1:',c_1)

# Name scopes add a prefix to all operations created in the same context.
with tf.name_scope("outer"):
    c_2 = tf.constant(2, name="c")  # => operation named "outer/c"
    print('c_2:',c_2)

    # Name scopes nest like paths in a hierarchical file system.
    with tf.name_scope("inner"):
        c_3 = tf.constant(3, name="c")  # => operation named "outer/inner/c"
        print('c_3:',c_3)

    # Exiting a name scope context will return to the previous prefix.
    c_4 = tf.constant(4, name="c")  # => operation named "outer/c_1"
    print('c_4:',c_4)

    # Already-used name scopes will be "uniquified".
    with tf.name_scope("inner"):
        c_5 = tf.constant(5, name="c")  # => operation named "outer/inner_1/c"
        print('c_5:',c_5)

c_0: Tensor("c:0", shape=(), dtype=int32)
c_1: Tensor("c_1:0", shape=(), dtype=int32)
c_2: Tensor("outer/c:0", shape=(), dtype=int32)
c_3: Tensor("outer/inner/c:0", shape=(), dtype=int32)
c_4: Tensor("outer/c_1:0", shape=(), dtype=int32)
c_5: Tensor("outer/inner_1/c:0", shape=(), dtype=int32)


#### 四. 执行图中的操作
1. 使用dict结构, 指定张量所依赖的placeholder张量
2. `sess.run()`中增加metadata参数, 来把计算过程存储到一个容器中, 以便回溯

In [ ]:
x = tf.placeholder(tf.float32,shape=[3]) #一维向量
y = tf.square(x)

with tf.Session() as sess:
    print(sess.run(y,{x:[1,2,3]}))

In [ ]:
y = tf.matmul([[37.0, -23.0], [1.0, 4.0]], tf.random_uniform([2, 2]))

with tf.Session() as sess:
    # Define options for the `sess.run()` call.
#     options = tf.RunOptions()
#     options.output_partition_graphs = True
#     options.trace_level = tf.RunOptions.FULL_TRACE

    # Define a container for the returned metadata.
#     metadata = tf.RunMetadata()

#     sess.run(y, options=options, run_metadata=metadata)
    sess.run(tf.global_variables_initializer())
    print(sess.run(y))

    # Print the subgraphs that executed on each device.
#     print(metadata.partition_graphs)

    # Print the timings of each operation that executed.
#     print(metadata.step_stats)

#### 五. 可视化计算图
`tf.summary.FileWriter("/tmp/log/testtf", sess.graph)`

In [ ]:
# Build your graph.
x = tf.constant([[37.0, -23.0], [1.0, 4.0]])
w = tf.Variable(tf.random_uniform([2, 2]))
y = tf.matmul(x, w)
# ...
loss = ...
train_op = tf.train.AdagradOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    # `sess.graph` provides access to the graph used in a <a href="../api_docs/python/tf/Session"><code>tf.Session</code></a>.
    writer = tf.summary.FileWriter("/tmp/log/testtf", sess.graph)

    # Perform your computation...
    for i in range(1000):
    sess.run(train_op)
    # ...

    writer.close()